In [2]:
import pandas as pd
import pymysql
conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='1234',db='sakila')
cursor = conn.cursor(pymysql.cursors.DictCursor)

### 앞으로 사용할 sakila data 목록 확인

In [3]:
sen = '''

show tables;

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,Tables_in_sakila
0,actor
1,actor_info
2,address
3,category
4,city
5,country
6,customer
7,customer_list
8,film
9,film_actor


In [4]:
sen = '''

DESC customer

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,Field,Type,Null,Key,Default,Extra
0,customer_id,smallint unsigned,NO,PRI,None,auto_increment
1,store_id,tinyint unsigned,NO,MUL,None,
2,first_name,varchar(45),NO,,None,
3,last_name,varchar(45),NO,MUL,None,
4,email,varchar(50),YES,,None,
5,address_id,smallint unsigned,NO,MUL,None,
6,active,tinyint(1),NO,,1,
7,create_date,datetime,NO,,None,
8,last_update,timestamp,YES,,CURRENT_TIMESTAMP,DEFAULT_GENERATED on update CURRENT_TIMESTAMP


### Select Clause & Column Aliases

In [13]:
### column 하나가지고 새로운 컬럼을 만들 수 있다.
### Column Aliases는 기존 table에 없던 새로운 column을 의미한다. as가 포함된 column이 이에 해당한다.

sen = '''

SELECT language_id,
    'COMMON' AS language_usage,
    language_id * 3.141592 AS language_pi_value,
    upper(name) AS language_name
FROM language;

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,language_id,language_usage,language_pi_value,language_name
0,1,COMMON,3.141592,ENGLISH
1,2,COMMON,6.283184,ITALIAN
2,3,COMMON,9.424776,JAPANESE
3,4,COMMON,12.566368,MANDARIN
4,5,COMMON,15.707960,FRENCH
5,6,COMMON,18.849552,GERMAN


In [12]:
sen = '''

SELECT version(),
    user(),
    database();

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,version(),user(),database()
0,8.0.28,root@localhost,sakila


### Removing Duplicates => pandas에서 unique method와 같다.

In [14]:
sen = '''
SELECT actor_id FROM film_actor ORDER BY actor_id;
'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,actor_id
0,1
1,1
2,1
3,1
4,1
...,...
5457,200
5458,200
5459,200
5460,200


SELECT 뒤에 DISTINCT를 추가하면 Distinct Set을 구할 수 있음.

In [15]:
sen = '''
SELECT DISTINCT actor_id FROM film_actor ORDER BY actor_id;
'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,actor_id
0,1
1,2
2,3
3,4
4,5
...,...
195,196
196,197
197,198
198,199


### From Clause

From은 테이블 여러 개를 불러 올 수 있다. 그중에 마음에 드는 columns을 불러와서 새로운 table을 만들 수 있다.

**Table 종류**
* Permanent tables => database에 저장된 테이블 
  
* Derived tables
* Temporary tables
* Virtual tables

**Derived tables**

In [16]:
sen = '''
SELECT concat(cust.last_name, ',', cust.first_name) full_name 
FROM
    (SELECT first_name, last_name, email
     FROM customer
     WHERE first_name = 'JESSIE'
    ) cust;
'''
### Fullname으로 column alias(가명이라는 의미)를 만든다. 
### cust라는 table은 parenthese의 내용으로 만들어진다. email은 왜 넣은거지 


cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,full_name
0,"BANKS,JESSIE"
1,"MILAM,JESSIE"


**Temporary tables**

In [17]:
sen = '''
CREATE TEMPORARY TABLE actors_j
    (
        actor_id smallint(5),
        first_name varchar(45),
        last_name varchar(45)
    );
'''
### Fullname으로 column alias(가명이라는 의미)를 만든다. 
### cust라는 table은 parenthese의 내용으로 만들어진다. email은 왜 넣은거지 


cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

""


In [18]:
sen = '''
INSERT INTO actors_j
SELECT actor_id, first_name, last_name
FROM actor
WHERE last_name LIKE 'J%';
'''
### actors_j라는 테이블에 actor라는 테이블에 있는 j가 맨앞에 있는 성을 가진 항목의 actor_id, first_name, last_name을 삽입하라.


cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

""


In [21]:
sen = '''
SELECT * FROM actors_j ORDER BY last_name
'''
### actors_j라는 테이블에 actor라는 테이블에 있는 j가 맨앞에 있는 성을 가진 항목의 actor_id, first_name, last_name을 삽입하라.


cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,actor_id,first_name,last_name
0,119,WARREN,JACKMAN
1,131,JANE,JACKMAN
2,8,MATTHEW,JOHANSSON
3,64,RAY,JOHANSSON
4,146,ALBERT,JOHANSSON
5,82,WOODY,JOLIE
6,43,KIRK,JOVOVICH
